In [18]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os

In [25]:
# API 설정
api_url = "http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList"
service_key = "EddYczqDlOv5/tU0oeG7NactEjZJJ77hlDl5ZfDyOzcJSAGZpGoSk9Frsq4ksUd9Cu30JAraDzx2cT7nKfMEqQ=="
num_of_rows = 100
page_no = 1

# 데이터 경로 설정
output_dir = "../data"
os.makedirs(output_dir, exist_ok=True)
output_csv = os.path.join(output_dir, "medicine_info.csv")

In [20]:
# 데이터프레임
columns = ["medicine_id", "name", "efficacy", "usage", "caution", "interaction", "sideEffect", "storageCondition"]
all_data = []

while True:
    params = {
        'serviceKey': service_key,
        'pageNo': page_no,
        'numOfRows': num_of_rows,
        'type': 'xml'
    }
    response = requests.get(api_url, params=params)
    response.encoding = 'utf-8'
    
    try:
        root = ET.fromstring(response.content)
    except ET.ParseError as e:
        print(f"Failed to parse XML: {e}")
        break
    
    # 아이템 추출
    items = root.findall(".//item")
    if not items:
        break
    
    for item in items:
        data = {
            "medicine_id": item.findtext("itemSeq"),
            "name": item.findtext("itemName"),
            "efficacy": item.findtext("efcyQesitm"),
            "usage": item.findtext("useMethodQesitm"),
            "caution": item.findtext("atpnQesitm"),
            "interaction": item.findtext("intrcQesitm"),
            "sideEffect": item.findtext("seQesitm"),
            "storageCondition": item.findtext("depositMethodQesitm")
        }
        all_data.append(data)
    
    page_no += 1

df = pd.DataFrame(all_data, columns=columns)

In [23]:
# CSV 파일로 저장
df.to_csv(output_csv, index=False, encoding='utf-8-sig')